In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("../data/youtoxic_english_1000.csv")

df.head()


In [ ]:
# Dimensiones del dataset
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

df.info()

print("\nValores nulos por columna:")
print(df.isnull().sum())


In [ ]:
# Distribución del target IsToxic
sns.countplot(x="IsToxic", data=df)
plt.title("Distribución de comentarios tóxicos vs no tóxicos")
plt.xlabel("¿Es tóxico?")
plt.ylabel("Número de comentarios")
plt.show()

# Porcentaje de cada clase
toxicity_ratio = df["IsToxic"].value_counts(normalize=True) * 100
print("Distribución porcentual:\n", toxicity_ratio)


In [ ]:
# Crear columnas nuevas para análisis de texto
df["text_length_chars"] = df["Text"].apply(len)
df["text_length_words"] = df["Text"].apply(lambda x: len(x.split()))

# Mostrar estadísticos por tipo de comentario
df.groupby("IsToxic")[["text_length_chars", "text_length_words"]].describe()


In [ ]:
# Boxplots para comparar longitud de texto por clase
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.boxplot(x="IsToxic", y="text_length_chars", data=df, ax=axes[0])
axes[0].set_title("Longitud en caracteres por tipo de comentario")

sns.boxplot(x="IsToxic", y="text_length_words", data=df, ax=axes[1])
axes[1].set_title("Longitud en palabras por tipo de comentario")

plt.tight_layout()
plt.show()


### Comparativa de longitud de los comentarios (Boxplot)

En esta visualización comparamos la **longitud de los comentarios tóxicos y no tóxicos**, tanto en número de caracteres como de palabras.

#### ¿Qué es un boxplot?
Un **boxplot** (o diagrama de caja) es una representación visual que nos permite ver:
- La **mediana** del conjunto de datos (línea central de la caja).
- El **rango intercuartílico** (donde se concentra el 50% de los valores).
- Los **valores extremos o outliers** (representados como puntos).

#### ¿Qué vemos en estos gráficos?
- La longitud de los comentarios **tóxicos y no tóxicos** es muy similar.
- Hay **muchos comentarios cortos** en ambos grupos.
- Aparecen **algunos comentarios muy largos** (outliers), especialmente en caracteres.
- No se observa una diferencia clara que nos permita decir que un tipo de comentario sea más largo que otro de forma sistemática.

#### ¿Qué concluimos?
Aunque puede haber pequeñas diferencias, **la longitud del comentario no parece ser un buen indicador por sí solo de si un comentario es tóxico o no**. Aun así, es útil conocer estas características para posibles decisiones de preprocesamiento, como filtrar comentarios excesivamente largos o cortos.


In [ ]:
# Filtrar comentarios tóxicos y no tóxicos
toxic_comments = df[df["IsToxic"] == True]["Text"]
nontoxic_comments = df[df["IsToxic"] == False]["Text"]

# Juntarlos en dos grandes textos para analizarlos
toxic_text = " ".join(toxic_comments).lower()
nontoxic_text = " ".join(nontoxic_comments).lower()


### Análisis del contenido textual

Para entender mejor las diferencias entre los comentarios tóxicos y no tóxicos, hemos separado los textos en dos grupos:

- Comentarios etiquetados como **tóxicos**.
- Comentarios etiquetados como **no tóxicos**.

Hemos unido los comentarios de cada grupo en un único texto para poder analizar qué palabras aparecen con mayor frecuencia en cada uno. Este enfoque nos permitirá visualizar patrones de lenguaje característicos, que luego pueden ser clave para entrenar un modelo predictivo.

En los próximos pasos generaremos:
- Listados de palabras más frecuentes.
- Nubes de palabras (*wordclouds*).
- N-gramas más comunes (combinaciones típicas de 2-3 palabras).


In [ ]:
from collections import Counter
import re

# Función para limpiar texto básico (sin lematizar aún)
def limpiar_texto(texto):
    texto = re.sub(r"[^\w\s]", "", texto)  # quitar signos de puntuación
    texto = texto.lower()  # pasar a minúsculas
    return texto

# Aplicar limpieza y dividir en palabras
palabras_toxicas = limpiar_texto(toxic_text).split()
palabras_no_toxicas = limpiar_texto(nontoxic_text).split()

# Contar palabras más comunes
frecuentes_toxicas = Counter(palabras_toxicas).most_common(10)
frecuentes_no_toxicas = Counter(palabras_no_toxicas).most_common(10)

# Mostrar resultados
print("🔴 Palabras más frecuentes en comentarios tóxicos:")
for palabra, freq in frecuentes_toxicas:
    print(f"{palabra}: {freq}")

print("\n🟢 Palabras más frecuentes en comentarios no tóxicos:")
for palabra, freq in frecuentes_no_toxicas:
    print(f"{palabra}: {freq}")


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

# Lista de stopwords en inglés
stop_words = set(stopwords.words("english"))

# Función mejorada que filtra stopwords
def limpiar_y_filtrar(texto):
    texto = re.sub(r"[^\w\s]", "", texto.lower())
    palabras = texto.split()
    return [p for p in palabras if p not in stop_words]

# Aplicar función mejorada
palabras_toxicas_filtradas = limpiar_y_filtrar(toxic_text)
palabras_no_toxicas_filtradas = limpiar_y_filtrar(nontoxic_text)

# Contar palabras más frecuentes (filtradas)
frecuentes_toxicas_filtradas = Counter(palabras_toxicas_filtradas).most_common(10)
frecuentes_no_toxicas_filtradas = Counter(palabras_no_toxicas_filtradas).most_common(10)

# Mostrar resultados
print("🔴 Palabras más frecuentes (tóxicos, sin stopwords):")
for palabra, freq in frecuentes_toxicas_filtradas:
    print(f"{palabra}: {freq}")

print("\n🟢 Palabras más frecuentes (no tóxicos, sin stopwords):")
for palabra, freq in frecuentes_no_toxicas_filtradas:
    print(f"{palabra}: {freq}")


### Análisis con eliminación de stopwords

Las palabras más frecuentes que observamos inicialmente eran muy comunes y poco informativas. Por ello, hemos repetido el análisis **eliminando las stopwords**, es decir, palabras muy frecuentes en inglés que no aportan significado real (como "the", "and", "is", etc.).

Esta limpieza **no forma aún parte del preprocesamiento oficial**, pero se introduce aquí como una forma de enriquecer el EDA y tomar decisiones más informadas.

Ahora los resultados empiezan a revelar **patrones de contenido más relevantes** para entender qué vocabulario podría distinguir los comentarios tóxicos de los no tóxicos.


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Unir las palabras ya filtradas
texto_toxico_filtrado = " ".join(palabras_toxicas_filtradas)
texto_nontoxico_filtrado = " ".join(palabras_no_toxicas_filtradas)

# Crear las nubes
wc_toxico = WordCloud(width=800, height=400, background_color="white").generate(texto_toxico_filtrado)
wc_nontoxico = WordCloud(width=800, height=400, background_color="white").generate(texto_nontoxico_filtrado)

# Mostrar
fig, axs = plt.subplots(1, 2, figsize=(18, 8))

axs[0].imshow(wc_toxico, interpolation="bilinear")
axs[0].axis("off")
axs[0].set_title("🔴 Comentarios tóxicos (sin stopwords)")

axs[1].imshow(wc_nontoxico, interpolation="bilinear")
axs[1].axis("off")
axs[1].set_title("🟢 Comentarios no tóxicos (sin stopwords)")

plt.tight_layout()
plt.show()


### WordClouds sin palabras vacías (stopwords)

Las siguientes nubes de palabras muestran los términos más repetidos en los comentarios **tóxicos** y **no tóxicos**, tras eliminar las palabras vacías típicas del inglés (como “the”, “and”, “is”…).

#### ¿Qué observamos?
- En los comentarios tóxicos aparecen con más frecuencia palabras como **“fuck”**, lo que indica un tono agresivo.
- También se observan términos raciales y relacionados con el orden público (**black, white, police**) en ambos grupos, lo que sugiere que el contexto es similar, pero el uso del lenguaje es lo que cambia.

Esta visualización permite a cualquier lector, incluso sin formación técnica, entender mejor el tipo de lenguaje que caracteriza cada grupo de comentarios.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Utilizamos los textos ya filtrados de stopwords
def mostrar_ngrams(textos, n=2, top=10):
    vectorizer = CountVectorizer(ngram_range=(n, n))
    X = vectorizer.fit_transform(textos)
    suma = X.sum(axis=0)
    freqs = [(word, suma[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    freqs_sorted = sorted(freqs, key=lambda x: x[1], reverse=True)
    return freqs_sorted[:top]

# Crear listas de texto plano por grupo
comentarios_toxicos = df[df["IsToxic"] == True]["Text"].tolist()
comentarios_no_toxicos = df[df["IsToxic"] == False]["Text"].tolist()

# Obtener n-gramas
top_bigrams_toxicos = mostrar_ngrams(comentarios_toxicos, n=2)
top_bigrams_nontoxicos = mostrar_ngrams(comentarios_no_toxicos, n=2)

top_trigrams_toxicos = mostrar_ngrams(comentarios_toxicos, n=3)
top_trigrams_nontoxicos = mostrar_ngrams(comentarios_no_toxicos, n=3)

# Mostrar resultados
print("🔴 Bigrams más comunes en comentarios tóxicos:")
for frase, freq in top_bigrams_toxicos:
    print(f"{frase}: {freq}")

print("\n🟢 Bigrams más comunes en comentarios no tóxicos:")
for frase, freq in top_bigrams_nontoxicos:
    print(f"{frase}: {freq}")

print("\n🔴 Trigrams más comunes en comentarios tóxicos:")
for frase, freq in top_trigrams_toxicos:
    print(f"{frase}: {freq}")

print("\n🟢 Trigrams más comunes en comentarios no tóxicos:")
for frase, freq in top_trigrams_nontoxicos:
    print(f"{frase}: {freq}")


### 🔗 Análisis de n-gramas (bigrams y trigrams)

Los **n-gramas** son combinaciones de palabras consecutivas que nos permiten identificar expresiones típicas o patrones de lenguaje. Son especialmente útiles en tareas como detección de discurso de odio, ya que muchas veces el contenido ofensivo se transmite en frases cortas y recurrentes.

#### ¿Qué buscamos aquí?
- Expresiones como “go back”, “shut up”, “you people” pueden tener fuerte carga ofensiva.
- Otras como “thank you” o “great video” pueden caracterizar comentarios positivos o neutrales.

Este análisis nos acerca a una comprensión más contextualizada del lenguaje usado, lo que será de gran valor a la hora de entrenar modelos de clasificación.


In [ ]:
import matplotlib.pyplot as plt

# Extraer trigramas y frecuencias por separado
trigrams_tox, freqs_tox = zip(*top_trigrams_toxicos)
trigrams_notox, freqs_notox = zip(*top_trigrams_nontoxicos)

# Crear gráfico de barras horizontales
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Trigramas tóxicos
axes[0].barh(trigrams_tox[::-1], freqs_tox[::-1], color="crimson")
axes[0].set_title("🔴 Trigramas más comunes en comentarios tóxicos")
axes[0].set_xlabel("Frecuencia")
axes[0].tick_params(axis='y', labelsize=10)

# Trigramas no tóxicos
axes[1].barh(trigrams_notox[::-1], freqs_notox[::-1], color="seagreen")
axes[1].set_title("🟢 Trigramas más comunes en comentarios no tóxicos")
axes[1].set_xlabel("Frecuencia")
axes[1].tick_params(axis='y', labelsize=10)

plt.tight_layout()
plt.show()


### Visualización de trigramas más frecuentes

Los trigramas (combinaciones de tres palabras consecutivas) nos permiten observar expresiones completas y típicas de cada tipo de comentario.

#### 🔴 En los comentarios tóxicos:
- Aparecen insultos explícitos como **“piece of shit”**.
- Expresiones violentas como **“run them over”** o **“shoot to apprehend”**.
- Términos relacionados con movimientos sociales cargados emocionalmente: **“black lives matter”**.

#### 🟢 En los comentarios no tóxicos:
- Predominan expresiones de agradecimiento (**“thank you for”**) o frases explicativas/descriptivas.
- Se nota un tono más **objetivo, civil o analítico**.

Este contraste deja ver claramente cómo cambia la intención del lenguaje entre ambos grupos y valida el uso de trigramas como posibles características valiosas para entrenar el modelo de detección de toxicidad.


In [ ]:
# Lista de etiquetas del dataset
etiquetas = [
    "IsToxic", "IsAbusive", "IsThreat", "IsProvocative", "IsObscene",
    "IsHatespeech", "IsRacist", "IsNationalist", "IsSexist", "IsHomophobic",
    "IsReligiousHate", "IsRadicalism"
]

# Calcular la correlación entre etiquetas
correlaciones = df[etiquetas].corr()

# Calcular cuántas etiquetas activas hay por comentario
df["TotalEtiquetas"] = df[etiquetas].sum(axis=1)
etiquetas_activas = df["TotalEtiquetas"].value_counts().sort_index()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualización de correlación entre etiquetas
plt.figure(figsize=(10, 8))
sns.heatmap(correlaciones, annot=True, cmap="coolwarm", fmt=".2f", vmin=-1, vmax=1)
plt.title("🔗 Correlación entre etiquetas de toxicidad")
plt.show()

# Visualización de número de etiquetas activas por comentario
etiquetas_activas.plot(kind="bar", color="steelblue", figsize=(8, 5))
plt.title("🎯 Número de etiquetas activas por comentario")
plt.xlabel("Cantidad de etiquetas")
plt.ylabel("Número de comentarios")
plt.xticks(rotation=0)
plt.grid(axis="y")
plt.show()


### Análisis cruzado de etiquetas

Además del análisis textual, es importante entender cómo se comportan las diferentes **subcategorías de toxicidad** que aparecen en el dataset (como `IsAbusive`, `IsThreat`, `IsRacist`, etc.).

#### ¿Cuántas etiquetas tiene cada comentario?

- Más de la mitad de los comentarios **no presentan ninguna etiqueta activa**.
- El resto puede tener múltiples etiquetas, lo que sugiere que **el problema podría ser tratado como multietiqueta** si quisiéramos predecir más allá de `IsToxic`.

#### ¿Qué etiquetas están relacionadas?

- `IsToxic` tiene una fuerte correlación con `IsAbusive` (**0.80**) y `IsProvocative`.
- `IsHatespeech` y `IsRacist` tienen una **correlación altísima** (**0.94**), lo que indica que suelen ir juntas.

Este análisis refuerza la idea de que la toxicidad **no es una dimensión única**, sino que puede tener varias expresiones combinadas. Si decidiéramos construir un modelo más complejo en el futuro, podríamos abordar este problema como una clasificación **multietiqueta** en lugar de binaria.
